In [72]:
import requests
import pandas as pd
import json
from math import ceil
from dotenv import load_dotenv
import os
import s3fs
import boto3

load_dotenv()

pd.set_option('display.max_rows', 10)
pd.set_option('mode.chained_assignment', None)

AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")
key = os.getenv("REDFIN_RAPIDAPI_KEY")

creds = {"key": AWS_ACCESS_KEY,
         "secret": AWS_SECRET_KEY}

In [73]:
# Variables
search_city1 = "Tulsa"
search_city2 = "Broken Arrow"
search_city3 = "Owasso"
search_city4 = "Bixby"
search_city5 = "Jenks"
search_state = "Oklahoma"
search_limit = 1000
# min_price_max_price = "50000,500000" # must be a string

# api variables unused currently
beds = 4
baths = 2
sqft = 1800 # minimum

# max iterative page count allowed
max_pages = 10


In [74]:
def return_regionId (city: str, limit: int, key: str):
	""" 
	Need to use this to find the regionId of the are you want to look 
	"""

	url = "https://redfin-com-data.p.rapidapi.com/properties/auto-complete"

	querystring = {"query": city,"limit": "50"}

	headers = {
		"X-RapidAPI-Key": key,
		"X-RapidAPI-Host": "redfin-com-data.p.rapidapi.com"
	}

	response = requests.get(url, headers=headers, params=querystring)

	return response.json()['data'][0]['rows'][0]['id']
	

TulsaRegionId = return_regionId(search_city1, 50, key)
BARegionId = return_regionId(search_city2, 50, key)
OwassoRegionId = return_regionId(search_city3, 50, key)
BixbyRegionId = return_regionId(search_city4, 50, key)
JenksRegionId = return_regionId(search_city5, 50, key)



In [75]:
RegionId_dict = {
                    "TulsaId" :     TulsaRegionId, 
                    "BAId" :        BARegionId, 
                    "OwassoId" :    OwassoRegionId, 
                    "BixbyId" :     BixbyRegionId, 
                    "JenksId" :     JenksRegionId
}

RegionId_dict

{'TulsaId': '6_35765',
 'BAId': '6_35693',
 'OwassoId': '6_14669',
 'BixbyId': '6_1686',
 'JenksId': '6_9591'}

In [76]:
for region in RegionId_dict.values():
    print(region)

6_35765
6_35693
6_14669
6_1686
6_9591


In [77]:
url = "https://redfin-com-data.p.rapidapi.com/properties/search-sale"
querystring = {
    "regionId": '6_9591',
    "limit": str(search_limit),
    "page": "1"
}
headers = {
    "X-RapidAPI-Key": key,
    "X-RapidAPI-Host": "redfin-com-data.p.rapidapi.com"
}
response = requests.get(url, headers=headers, params=querystring)
json_data = response.json()


In [78]:
json_list = []

for region in RegionId_dict.values():
    
    url = "https://redfin-com-data.p.rapidapi.com/properties/search-sale"
    querystring = {
        "regionId": f"{region}",
        "limit": f"{search_limit}",
        "page": "1",
    }
    headers = {
        "X-RapidAPI-Key": key,
        "X-RapidAPI-Host": "redfin-com-data.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code != 200:
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)
    else:
        json_data = response.json()
        json_list.append(json_data['data'])
        print(f'json_list length {len(json_list)}')
        print(f'page 1 query = {querystring}')
        print(json_data['meta']['moreData'])

    while json_data['meta']['moreData'] == True:
        pg_num = str(int(json_data['meta']['currentPage']) + 1)
        url = "https://redfin-com-data.p.rapidapi.com/properties/search-sale"
        querystring = {
            "regionId": f"{region}",
            "limit": str(search_limit),
            "page": pg_num,
        }
        headers = {
            "X-RapidAPI-Key": key,
            "X-RapidAPI-Host": "redfin-com-data.p.rapidapi.com"
        }
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code != 200:
            print(f"Request failed with status code: {response.status_code}")
            print(response.text)
        else:
            json_data2 = response.json()
            json_list.append(json_data2['data'])
            print(f'json_list length {len(json_list)}')
            print(f'page {pg_num} query = {querystring}')
            json_data = json_data2  # Update json_data with the new response

    print(f'{region} loop complete')
print(len(json_list))

json_list length 1
page 1 query = {'regionId': '6_35765', 'limit': '1000', 'page': '1'}
True
json_list length 2
page 2 query = {'regionId': '6_35765', 'limit': '1000', 'page': '2'}
6_35765 loop complete
json_list length 3
page 1 query = {'regionId': '6_35693', 'limit': '1000', 'page': '1'}
False
6_35693 loop complete
json_list length 4
page 1 query = {'regionId': '6_14669', 'limit': '1000', 'page': '1'}
False
6_14669 loop complete
json_list length 5
page 1 query = {'regionId': '6_1686', 'limit': '1000', 'page': '1'}
False
6_1686 loop complete
json_list length 6
page 1 query = {'regionId': '6_9591', 'limit': '1000', 'page': '1'}
False
6_9591 loop complete
6


In [79]:
"""
Now we can loop through the list and populate a dictionary that will be converted to a DF
"""

home_list = []
for i in range(0,len(json_list)):
    home_iteration_list = json_list[i]            
    for home in home_iteration_list:
        home_data = home['homeData']
        home_list.append({
            "property_id": home_data['propertyId'],
            "listing_id": home_data['listingId'],
            "street_address": home_data['addressInfo'].get('formattedStreetLine',''),
            "city": home_data['addressInfo']['city'],
            "state": home_data['addressInfo']['state'],
            "zip": home_data['addressInfo']['zip'],
            "price": home_data['priceInfo'].get('amount',''),
            "beds": home_data.get('beds',''),
            "baths": home_data.get('baths',''),
            "sqft": home_data.get('sqftInfo','').get('amount',''),
            "lot": home_data['lotSize'].get('amount', ''),
            "year_built": home_data['yearBuilt'].get('yearBuilt','New Build'),
            "days_on_market": home_data['daysOnMarket']['daysOnMarket'],
            "data_added": home_data['daysOnMarket']['listingAddedDate'],
            "hoa_dues": home_data['hoaDues'].get('amount', ''),
            "listing_url": 'https://www.redfin.com' + home_data['url']
        })

home_df = pd.DataFrame(home_list)
home_df


,property_id,listing_id,street_address,city,state,zip,price,beds,baths,sqft,lot,year_built,days_on_market,data_added,hoa_dues,listing_url
0,74255243,190742400,5810 S New Haven Ave,Tulsa,OK,74135,515000,4,4.0,3362,11264,1965,1,2024-07-05T20:29:15.332Z,,https://www.redfin.com/OK/Tulsa/5810-S-New-Hav...
1,74323002,190738431,1223 S Lawton Ave #12,Tulsa,OK,74127,225000,2,2.5,1643,,1982,1,2024-07-05T18:51:39.888Z,190,https://www.redfin.com/OK/Tulsa/1223-S-Lawton-...
2,74324517,190069254,26 S Tacoma Ave,Tulsa,OK,74127,375000,3,2.5,2761,17875,1912,17,2024-06-20T05:09:00.164Z,,https://www.redfin.com/OK/Tulsa/26-S-Tacoma-Av...
3,74384544,188548126,7640 S Quebec Pl,Tulsa,OK,74136,375000,3,2.5,2554,13677,1976,18,2024-05-29T19:06:55.700Z,,https://www.redfin.com/OK/Tulsa/7640-S-Quebec-...
4,74390634,188786848,9207 E 68th St,Tulsa,OK,74133,267000,4,2.0,2111,9612,1978,37,2024-05-30T22:38:25.561Z,25,https://www.redfin.com/OK/Tulsa/9207-E-68th-St...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,188380697,177539214,3316,Jenks,OK,74037,581500,5,3.5,3395,,New Build,241,2023-11-03T21:52:46.817Z,54,https://www.redfin.com/OK/Jenks/Jenks/3316/hom...
3385,188383588,177544840,Williams One L,Jenks,OK,74037,509900,3,2.5,2605,,New Build,241,2023-11-03T21:53:46.636Z,54,https://www.redfin.com/OK/Jenks/Jenks/Williams...
3386,188378294,177534572,3769,Jenks,OK,74037,627500,4,3.5,3850,,New Build,241,2023-11-03T21:52:01.530Z,54,https://www.redfin.com/OK/Jenks/Jenks/3769/hom...
3387,188376647,177531438,Morgan L,Jenks,OK,74037,576900,4,3.5,3349,,New Build,241,2023-11-03T21:51:19.201Z,54,https://www.redfin.com/OK/Jenks/Jenks/Morgan-L...


In [80]:
home_df.to_csv("s3://redfin-project-bucket/data/redfin-listings-csv/redfin_home_listings.csv", index=False, storage_options=creds)